In [1]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem.porter import PorterStemmer
porter_stemmer  = PorterStemmer()
import re
import inflect
stop_words=set(stopwords.words('english'))
def Pre_Processing(file):
    token_files=[] 
    after_lower=[]
    after_lemmatizer=[]
    after_stemming=[]
    tokenizer = RegexpTokenizer(r'\w+')
    tokens=(tokenizer.tokenize(file))
    for i in range(len(tokens)):
        after_lower.append(tokens[i].lower())
    after_lower=[x for x in after_lower if x not in stop_words ]
    for i in range(len(after_lower)):
        #after_lemmatizer.append(wordnet_lemmatizer.lemmatize(after_lower[i]))
        after_stemming.append(porter_stemmer.stem(after_lower[i]))
    return after_stemming

In [2]:
import os
import math
import codecs
TF_Dictionary={}
Docid=[]
parent="C:/Users/Devashi Jain/Desktop/IIIT-D/Information Retrieval/Assignment4/Classes"
for file_name in os.listdir(os.path.join(parent)): 
    for file in os.listdir(os.path.join(parent,file_name)):
        doc=file_name+'/'+file
        Docid.append(doc)
        preprossed_file=[]
        Normalized_TF={}
        
        fd=codecs.open(parent+'/'+file_name+'/'+file,'r',errors='ignore',encoding='utf-8')
        preprossed_file=Pre_Processing(fd.read())
        for i in range(len(preprossed_file)):

            if preprossed_file[i] in Normalized_TF:
                Normalized_TF[preprossed_file[i]]+=1
            else:
                Normalized_TF[preprossed_file[i]]=1


        for i in range(len(preprossed_file)):
            if preprossed_file[i] in TF_Dictionary:
                TF_Dictionary[preprossed_file[i]][doc]=Normalized_TF[preprossed_file[i]]/len(preprossed_file)
            else:
                TF_Dictionary[preprossed_file[i]] = {doc:Normalized_TF[preprossed_file[i]]/len(preprossed_file)}



In [3]:
print(len(TF_Dictionary))

56899


In [4]:
import math
for term in TF_Dictionary:
    IDF=1+math.log(5000/len(TF_Dictionary[term]))
    for file in TF_Dictionary[term]:
        TF_Dictionary[term][file]*=IDF

In [ ]:
TF_IDF_Dictionary={}
for term in TF_Dictionary:
    t=0
    for doc in TF_Dictionary[term]:
        t=t+TF_Dictionary[term][doc]
    TF_IDF_Dictionary[term]=t

In [ ]:
Top_Features={}
n=int(len(TF_IDF_Dictionary)*0.1)
t = sorted(TF_IDF_Dictionary.items(), key=lambda x:-x[1])[:n]
for i in range(len(t)):
    Top_Features[t[i][0]]=t[i][1]

In [ ]:
print(len(Top_Features))

In [5]:
import os
import math
import codecs

Doc_Vec={}
for doc1 in Docid:
    c=0
    for term in TF_Dictionary.keys():
        if doc1 not in Doc_Vec:
            Doc_Vec.setdefault(doc1,{})
            Doc_Vec[doc1]={term:0}
            c=c+1
        else:
            Doc_Vec[doc1][term]=0
            c=c+1

In [6]:
import math
parent="C:/Users/Devashi Jain/Desktop/IIIT-D/Information Retrieval/Assignment4/Classes"
cl=[]
for file_name in os.listdir(os.path.join(parent)): 
    for file in os.listdir(os.path.join(parent,file_name)):
        doc=file_name+'/'+file
        preprossed_file=[]
        Normalized_TF={}

        cl.append(doc)
        fd=codecs.open(parent+'/'+file_name+'/'+file,'r',errors='ignore',encoding='utf-8')
        preprossed_file=Pre_Processing(fd.read())
        for i in range(len(preprossed_file)):

            if preprossed_file[i] in Normalized_TF:
                Normalized_TF[preprossed_file[i]]+=1
            else:
                Normalized_TF[preprossed_file[i]]=1

        for term in Doc_Vec[doc]:
#                 IDF=1+math.log(3500/len(TF_Dictionary[term]))
            if term in Normalized_TF:
                Doc_Vec[doc][term]=(Normalized_TF[term])



In [7]:
print(len(Doc_Vec))

5000


In [8]:
import scipy.spatial.distance as metric
def computeDist(A,B):
    return metric.euclidean(A, B)

In [ ]:
def Calculate_RSS(Data,centroids,clusters):
    rss=0
    for term in Data:
        for i in range(5):
            if term in clusters[i]:
                list1=list(Data[term].values())
                list2=list(centroids[i].values())
                s=computeDist(list1,list2)
                rss+=s
                break
    return rss

In [ ]:
def Assign_Cluster(Data,Centroids):
    clusters=[[] for i in range(k)]
    sums=[[] for i in range(k)]
    for term in Data:
        x=term.split('/')
        list1=list(Data[term].values())
        dist=[0 for i in range(k)]
        for l in range(k):
            list2=list(Centroids[l].values())
            dist[l]=computeDist(list1,list2)
        minI=np.argmin(dist)
        clusters[minI].append(term)
        c[minI].append(x[0])
        for word in docs[term]:
            if word in sums[minI]:
                sums[minI][word]+=Data[term][word]
            else:
                sums[minI][word]=Data[term][word]
    return sums,clusters

In [ ]:
def Centroid_Updation(sums,clusters):
    centroids=[[] for i in range(k)]
    for i in range(k):        
            centroids[i] = {k: v / len(clusters[i]) for k, v in sums[i].items()}
    return  centroids 

In [9]:
def kmeans(docs,k):
    centro=random.sample( docs.items(), k )
    centroids=[centro[i][1] for i in range(k)]
    print("Initial Centroids Computed")
    clusters=[[] for i in range(k)]
    counter=0
    while(True):
        prevCentroids=copy.deepcopy(centroids)
        clusters=[[] for i in range(k)]
        sums=[{} for i in range(k)]
        
        print("Assigning Clusters")
        sums,clusters=Assign_Cluster(docs,centroids)
        
        print("Centroid Updation")
        centroids=Centroid_Updation(sums,clusters)
        
        fprint("Calculating RSS")
        RSS=Calculate_RSS(docs,centroids,clusters)
        
        print("Iteration ",counter," done")
        if(counter>50):
            print("Max iteration reached")
            break
        flag=False
        for cent in centroids:
            if cent not in prevCentroids:
                flag=False
                break
            else:
                flag=True
        if(flag==True):
            print("Converged by same centroids")
            break
        print("RSS: ",RSS)
        counter+=1
    return clusters,centroids


In [11]:
import random
import copy
import numpy as np
cluster,cente=kmeans(Doc_Vec,5)

Initial Centroids Computed
Cluster Assignment
Centroids recomputation
Iteration  0  done
RSS:  123209.78452706679
Cluster Assignment
Centroids recomputation
Iteration  1  done
RSS:  119858.42573451044
Cluster Assignment
Centroids recomputation
Iteration  2  done
RSS:  118920.23829753797
Cluster Assignment
Centroids recomputation
Iteration  3  done
RSS:  118692.63812001544
Cluster Assignment
Centroids recomputation
Iteration  4  done
RSS:  118556.50648281173
Cluster Assignment
Centroids recomputation
Iteration  5  done
RSS:  118546.94096735153
Cluster Assignment
Centroids recomputation
Iteration  6  done
RSS:  118502.76898131671
Cluster Assignment
Centroids recomputation
Iteration  7  done
RSS:  118423.1869258269
Cluster Assignment
Centroids recomputation
Iteration  8  done
RSS:  118332.50951127932
Cluster Assignment
Centroids recomputation
Iteration  9  done
RSS:  117982.57906314383
Cluster Assignment
Centroids recomputation
Iteration  10  done
RSS:  117672.56240731604
Cluster Assignme

In [12]:
f=[]
for i in range(len(cluster)):
    c=[]
    for term in cluster[i]:
        x=term.split('/')
        c.append(x[0])
    f.append(c)

In [13]:
def purity(cluster):
    purity=np.zeros(5)
    list1=['comp.graphics',
    'rec.sport.hockey',
    'sci.med',
    'sci.space',
    'talk.politics.misc']
    for i in range(len(cluster)):
        x=[]
        for term in list1:
            print(term)
            x.append(cluster[i].count(term))
        print(x)
        maxi=max(x)
        purity[i]=maxi
        print(purity[i])
        
    s=0
    for i in range(len(purity)):
        s=s+purity[i]
    return s/5000

In [14]:
Purity=purity(f)

comp.graphics
rec.sport.hockey
sci.med
sci.space
talk.politics.misc
[0, 2, 0, 0, 0]
2.0
comp.graphics
rec.sport.hockey
sci.med
sci.space
talk.politics.misc
[0, 0, 8, 4, 11]
11.0
comp.graphics
rec.sport.hockey
sci.med
sci.space
talk.politics.misc
[10, 0, 0, 0, 0]
10.0
comp.graphics
rec.sport.hockey
sci.med
sci.space
talk.politics.misc
[0, 22, 0, 0, 0]
22.0
comp.graphics
rec.sport.hockey
sci.med
sci.space
talk.politics.misc
[990, 976, 992, 996, 989]
996.0


In [15]:
Purity

0.2082

In [16]:
import pandas as pd
def ARI(cluster):
    ARI1=[]
    list1=['comp.graphics',
    'rec.sport.hockey',
    'sci.med',
    'sci.space',
    'talk.politics.misc']
    for i in range(len(cluster)):
        x=[]
        for term in list1:
            x.append(cluster[i].count(term))
        print(x)
        ARI1.append(x)
    Index=0
    Expected_Index=0
    Max_Index=0
    for i in range(len(ARI1)):
        for j in range(len(ARI1[i])):
            if(ARI1[i][j]>=2):
                s=(ARI1[i][j]*(ARI1[i][j]-1))/2
                Index=Index+s
    A=0
    B=0
    for i in range(len(ARI1)):
        a=sum(ARI1[i])
        if(a>=2):
            s=(a*(a-1))/2
            A=A+s
    df=pd.DataFrame(ARI1)
    for i in range(len(df)):
        b=sum(df[i])
        if(b>=2):
            s=(b*(b-1))/2
            B=B+s
    Expected_Index= (A*B)/300
    Max_Index=0.5*(A+B)
    ari=(Index-Expected_Index)/(Max_Index-Expected_Index)
    return ari
Adjusted_Rand_Index=ARI(f)
print(Adjusted_Rand_Index)

[0, 2, 0, 0, 0]
[0, 0, 8, 4, 11]
[10, 0, 0, 0, 0]
[0, 22, 0, 0, 0]
[990, 976, 992, 996, 989]
1.0000483356048968
